<a href="https://colab.research.google.com/github/RO-AD/waymo-od-motion-pred/blob/main/hj_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Attention is All You Need (NIPS 2017) 실습
- 아래 링크 보고 transformer 모델 만들어보기
https://github.com/hyojinlee98/College/blob/master/tutorial/1_transformer/hj_transformer.ipynb

- 코드 실행 전에 [런타임] → [런타임 유형 변경] → 유형을 GPU로 설정

## BLEU Score 계산을 위한 라이브러리 업데이트
- [Restart Runtime] 버튼을 눌러 런타임을 재시작
- `bleu_score` 모듈이 있는 torchtext 라이브러리 설치 
  - from torchtext.data.metrics import bleu_score

In [1]:
%%capture
!pip install torchtext==0.6.0

## 데이터 전처리 (Preprocessing)

NLP에서 텍스트 전처리 작업이 필요하다. 텍스트 전처리를 위해서는 클렌징, 토큰화, 불용어 제거, 정규화 등의 작업이 필요하다. 이번에는 토큰화를 진행할 것이다. 토큰(Token)이란 문법적으로 더 이상 나눌 수 없는 언어 요소를 뜻한다. 텍스트 토큰화란 말뭉치로부터 토큰을 분리하는 작업을 뜻한다. 문장 토큰화와 단어 토큰화가 있는데, 이번에 사용하는 라이브러리는 단어 토큰화를 해볼 것이다. 기본적으로 띄어쓰기를 기준으로 하고, 콤마와 마침표는 별도의 토큰으로 구분한다. `Don't` 같은 경우는 `Do`와 `n't`로 분리된다.

- spaCy 라이브러리 : 문장의 토큰화(tokenization), 태깅(tagging) 등의 전처리 기능을 위한 라이브러리
  - 영어(English)와 한국어(Korea) 전처리 모듈 설치

In [2]:
# %%는 Colab의 매직 커맨드 (결과 출력 X / 뒤에 이름 지정해주면 지정 파일에 결과값 저장)
%%capture 
!python -m spacy download en_core_web_sm
!python -m spacy download ko_core_news_sm # ko가 없음

In [3]:
import spacy

spacy_en = spacy.load('en_core_web_sm') # 영어 토큰화(tokenization)
spacy_ko = spacy.load('ko_core_news_sm') # 한국어 토큰화(tokenization)

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [4]:
# 간단히 영어 토큰화 기능 써보기 - 단어로 토큰화 
tokenized = spacy_en.tokenizer("I am a graduate student.")

for i, token in enumerate(tokenized) :
  print(f'인덱스 {i} : {token.text}')

인덱스 0 : I
인덱스 1 : am
인덱스 2 : a
인덱스 3 : graduate
인덱스 4 : student
인덱스 5 : .


In [5]:
# 간단히 한국어 토큰화 기능 써보기 - 단어로 토큰화
tokenized = spacy_ko.tokenizer("나는 대학원생이다.")

for i, token in enumerate(tokenized) :
  print(f"인덱스 {i} : {token.text}")

인덱스 0 : 나는
인덱스 1 : 대학원생이다
인덱스 2 : .


- 영어(English) 및 한국어(Korea) 토큰화 함수 정의

In [6]:
# 한국어 문장을 토큰화 하는 함수 (순서를 뒤집지 않음)
def tokenize_ko(text) :
  return [token.text for token in spacy_ko.tokenizer(text)]

# 영어 문장을 토큰화 하는 함수
def tokenize_en(text) :
  return [token.text for token in spacy_en.tokenizer(text)]

- 필드(field) 라이브러리를 이용해 데이터셋에 대한 구체적인 전처리 내용을 명시한다.
- Seq2Seq 모델과는 다르게 batch_first 속성의 값을 True로 설정한다.
- 번역 목표 
  - 소스(SRC) : 한국어
  - 목표(TRG) : 영어

In [7]:
from torchtext.data import Field, BucketIterator

# sos : Start Of Statement
# eos : End Of Statement
# batch_first : input 의 shape 에서 batch_size 를 shape 의 첫번째에 위치할 수 있도록 설정
SRC = Field(tokenize=tokenize_ko, init_token="<sos>", eos_token="<eos>", lower=True, batch_first=True)
TRG = Field(tokenize=tokenize_en, init_token="<sos>", eos_token="<eos>", lower=True, batch_first=True)

- 원래 예제는 영어-독일어 번역 모델을 만드는 거라 대표적인 영어-독어 번역 데이터셋인 **torchtext.datasets.Multi30k**를 사용했다. 
- 영어-한국어 번역 모델을 만드는 게 목표이기 때문에 AI hub에 있는 번역 데이터셋인 Korpora 설치

In [8]:
!pip install Korpora

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


- Korpora 데이터셋 종류

In [9]:
from Korpora import Korpora
Korpora.corpus_list()

{'kcbert': 'beomi@github 님이 만드신 KcBERT 학습데이터',
 'korean_chatbot_data': 'songys@github 님이 만드신 챗봇 문답 데이터',
 'korean_hate_speech': '{inmoonlight,warnikchow,beomi}@github 님이 만드신 혐오댓글데이터',
 'korean_parallel_koen_news': 'jungyeul@github 님이 만드신 병렬 말뭉치',
 'korean_petitions': 'lovit@github 님이 만드신 2017.08 ~ 2019.03 청와대 청원데이터',
 'kornli': 'KakaoBrain 에서 제공하는 Natural Language Inference (NLI) 데이터',
 'korsts': 'KakaoBrain 에서 제공하는 Semantic Textual Similarity (STS) 데이터',
 'kowikitext': 'lovit@github 님이 만드신 wikitext 형식의 한국어 위키피디아 데이터',
 'namuwikitext': 'lovit@github 님이 만드신 wikitext 형식의 나무위키 데이터',
 'naver_changwon_ner': '네이버 + 창원대 NER shared task data',
 'nsmc': 'e9t@github 님이 만드신 Naver sentiment movie corpus v1.0',
 'question_pair': 'songys@github 님이 만드신 질문쌍(Paired Question v.2)',
 'modu_news': '국립국어원에서 만든 모두의 말뭉치: 뉴스 말뭉치',
 'modu_messenger': '국립국어원에서 만든 모두의 말뭉치: 메신저 말뭉치',
 'modu_mp': '국립국어원에서 만든 모두의 말뭉치: 형태 분석 말뭉치',
 'modu_ne': '국립국어원에서 만든 모두의 말뭉치: 개체명 분석 말뭉치',
 'modu_spoken': '국립국어원에서 만든 모두의 말뭉치: 구어 

In [36]:
Korpora.fetch('korean_parallel_koen_news', root_dir='/content/Korpora')

[korean_parallel] download korean-english-park.train.tar.gz: 8.72MB [00:00, 63.4MB/s]                            


decompress /content/Korpora/korean_parallel/korean-english-park.train.tar.gz


[korean_parallel] download korean-english-park.dev.tar.gz: 115kB [00:00, 1.88MB/s]


decompress /content/Korpora/korean_parallel/korean-english-park.dev.tar.gz


[korean_parallel] download korean-english-park.test.tar.gz: 238kB [00:00, 3.66MB/s]

decompress /content/Korpora/korean_parallel/korean-english-park.test.tar.gz


In [56]:
!ls /content/Korpora/korean_parallel

korean-english-park.dev.en	korean-english-park.test.tar.gz
korean-english-park.dev.ko	korean-english-park.train.en
korean-english-park.dev.tar.gz	korean-english-park.train.ko
korean-english-park.test.en	korean-english-park.train.tar.gz
korean-english-park.test.ko


In [62]:
datasets = Korpora.load('korean_parallel_koen_news', root_dir='/content/Korpora')
datasets


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : KakaoBrain
    Repository : https://github.com/jungyeul/korean-parallel-corpora
    References :
        - Jungyeul Park, Jeen-Pyo Hong and Jeong-Won Cha (2016) Korean Language Resources for Everyone.
          In Proceedings of the 30th Pacific Asia Conference on Language, Information and Computation
          (PACLIC 30). October 28 - 30, 2016. Seoul, Korea. 
          (https://www.aclweb.org/anthology/Y16-2002/)

    # License
    Creative Commons Attribution Noncommercial No-Derivative-Works 3.0
    Details in https://creativecommons.org/licenses/by-nc-nd/3.0/



In [72]:
datasets.train

SentencePair(text='개인용 컴퓨터 사용의 상당 부분은 "이것보다 뛰어날 수 있느냐?"', pair='Much of personal computing is about "can you top this?"')

In [135]:
import pandas as pd

In [136]:
train_ko_df = pd.DataFrame(datasets.train.texts, columns=['src'])
train_en_df = pd.DataFrame(datasets.train.pairs, columns=['trg'])
train_df = train_ko_df.join(train_en_df)
train_df.head()

,src,trg
0,"개인용 컴퓨터 사용의 상당 부분은 ""이것보다 뛰어날 수 있느냐?""","Much of personal computing is about ""can you t..."
1,모든 광마우스와 마찬가지 로 이 광마우스도 책상 위에 놓는 마우스 패드를 필요로 하...,so a mention a few weeks ago about a rechargea...
2,그러나 이것은 또한 책상도 필요로 하지 않는다.,"Like all optical mice, But it also doesn't nee..."
3,"79.95달러하는 이 최첨단 무선 광마우스는 허공에서 팔목, 팔, 그외에 어떤 부분...",uses gyroscopic sensors to control the cursor ...
4,정보 관리들은 동남 아시아에서의 선박들에 대한 많은 (테러) 계획들이 실패로 돌아갔...,Intelligence officials have revealed a spate o...


In [137]:
test_ko_df = pd.DataFrame(datasets.test.texts, columns=['src'])
test_en_df = pd.DataFrame(datasets.test.pairs, columns=['trg'])
test_df = test_ko_df.join(test_en_df)
test_df.head()

,src,trg
0,토론에 참여한 사람들은 법 집행과 국가 안전보장에 대한 우려를 표명해야 할 필요성을...,Those involved in the discussions do take seri...
1,"또한 새로운 기술 개발이 어떤 해결책을 제공해 주는데 도움이 될 것이고, 동시 에 ...",There is also some hope that new technology de...
2,그래서 클리퍼 칩에 대한 개인적인 해결책은 서서히 자취를 감출 것입니다. 그러나 이...,So the individual solution of the clipper chip...
3,이탈리아의 천문학자들이 멀리에 있는 별들의 궤도를 도는 행성의 대기에서 생명체의 필...,"Italian astronomers have found signs of water,..."
4,"물이 있다는 것이 다른 행성에 생명체들이 가득하다는 것을 의미하지는 않지만, 이번 ...",Having water does not mean other planets will ...


In [160]:
train_df.to_csv('/content/Korpora/korean_parallel/train.csv', index=False)
test_df.to_csv('/content/Korpora/korean_parallel/test.csv', index=False)

In [89]:
from torchtext.data import TabularDataset

In [162]:
train_dataset, test_dataset = TabularDataset.splits(path='/content/Korpora/korean_parallel', 
                                              train='train.csv', test='test.csv', skip_header=True,
                                              format='csv', fields = [('src', SRC), ('trg', TRG)])

In [166]:
train_dataset.examples[0].trg

['much',
 'of',
 'personal',
 'computing',
 'is',
 'about',
 '"',
 'can',
 'you',
 'top',
 'this',
 '?',
 '"']

In [ ]:
print(f"학습 데이터셋(training dataset) 크기: {len(train_dataset.examples)}개")
print(f"테스트 데이터셋(testing dataset) 크기: {len(test_dataset.examples)}개")